In [ ]:
# https://sites.google.com/site/ryunosukehm/study/ml-with-python/impact-ss2017/hmm_toy_example
# https://www.kabuku.co.jp/developers/hmm
# http://keik-117.hatenablog.com/entry/2016/07/05/213903
import numpy as np
from hmmlearn import hmm

In [ ]:
# https://hmmlearn.readthedocs.io/en/latest/api.html#multinomialhmm
# 隠れた状態は指番号 観測できるのは音符列
model = hmm.MultinomialHMM(n_components=5)

model.startprob_ = np.array([0.2, 0.2, 0.2, 0.2, 0.2])
# ある指番号から指番号への遷移確率
model.transmat_ = np.array([
    [0.00, 0.50, 0.40, 0.05, 0.05],
    [0.30, 0.00, 0.60, 0.05, 0.05],
    [0.15, 0.50, 0.00, 0.30, 0.05],
    [0.15, 0.10, 0.50, 0.00, 0.25],
    [0.20, 0.05, 0.05, 0.70, 0.00]])
# 出力を単純化 上行 = 2, 下行 = 1
model.emissionprob_ = np.array([
    [0.00, 0.20, 0.80],
    [0.00, 0.40, 0.60],
    [0.00, 0.60, 0.40],
    [0.00, 0.75, 0.25],
    [0.00, 0.90, 0.10]])

In [ ]:
X, Z = model.sample(10)
Z + 1

In [ ]:
model.predict(np.array([
    [2],  # C -> D
    [2],  # D
    [2],  # E
    [1],  # F
    [2],  # D
    [1],  # E
    [2]   # C
])) + 1

In [ ]:
import music21
s = music21.converter.parse('./mxl/note_example.xml')
notes = s.flat.notes
midies = [note.pitch.midi for note in notes]
differs = [midies[i + 1] - midies[i] for i in range(len(midies) - 1)]
outputs = [2 if d > 0 else 1 for d in differs]
outputs.append(1)
outputs

In [ ]:
model.predict(np.array(outputs).reshape(len(outputs), 1)) + 1